In [1]:
import PyMagmaCh.A1_domain.domain as dmn
import time, copy
import numpy as np
from PyMagmaCh.A1_domain.field import Field
from PyMagmaCh.A1_domain.domain import _Domain
from PyMagmaCh.utils import constants as const
from PyMagmaCh.process.process import Process,get_axes
from PyMagmaCh.process.time_dependent_process import TimeDependentProcess
from PyMagmaCh.utils.walk import walk_processes


In [2]:
z_clmn, atm_slab =dmn.z_column_atm()
z_clmn.name = 'z_clmn'
atm_slab.name = atm_slab.domain_type

A = np.linspace(1,10,10)
a1 = Field(A,domain = z_clmn,axis = {'1':z_clmn.axes['depth'].axis_type,'2':z_clmn.axes['depth'].axis_type})
a1.name='a1'

B = np.linspace(1,120,10)
b1 = Field(B,domain = z_clmn,axis = z_clmn.axes['depth'].axis_type)
b1.name='b1'



In [3]:
a2 = Process(state=a1, domains=z_clmn, name_inp='a2')
a3 = TimeDependentProcess(state=a1, domains=z_clmn, name_inp='a3')

/home/tmittal/mantle_plumes/magma_chamber_model/Version2/PyMagmaCh/process/process.py:53: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if (state != None):


In [4]:
import numpy as np
from scipy import integrate
from PyMagmaCh.process.time_dependent_process import TimeDependentProcess
from PyMagmaCh.utils import constants as const
from PyMagmaCh.utils import model_degruyter as md_deg
from PyMagmaCh.A1_domain.field import Field

class Chamber_model(TimeDependentProcess):
    '''
    Parent class for box model of a magma chamber -
    Typically solve for P,V,T + other things
    Sequence - solve coupled ode's for the variables
    '''
    def __init__(self,chamber_shape=None,**kwargs):
        super(Chamber_model, self).__init__(**kwargs)
        self.process_type = 'explicit'
        self.chamber_shape = chamber_shape
        # this keeps track of variables to solve later, default is false for all state var
        self.solve_me ={}  
        for varname, value in self.state.items():
            self.solve_me.update({varname: False})

    def func_ode(self,t,X, arg1):
        '''Specify the coupled ode functions to integrate forward ..
        This method should be over-ridden by daughter classes.'''
        pass

    def ode_solver(self):
        '''Need to specify - solve_me state variable in the daughter classes.
        Also need to make the state variables a function of time
        (i.e. start at t = 0, then move append value in the field)
        Need to be careful of the sequence in state.items()
        '''
        max_t = self.param['timestep']      # Total integration time (seconds)
        X_init = np.array([])
        X_init_var = np.array([])
        for varname, value in self.state.items():
            try:
                if (self.solve_me[varname] == True):
                    tmp1 = self.state[varname][-1]
                    X_init = np.append(X_init,tmp1)
                    X_init_var = np.append(X_init_var,varname)
            except:
                pass
        tmp2 = integrate.ode(func_ode).set_integrator('dopri5',nsteps=1e8)
        tmp2.set_initial_value(X_init,0.0).set_f_params(X_init_var)
        X_new = tmp2.integrate(tmp2.t+max_t)
        state_var_update_func(X_new)

    def state_var_update_func(self,X_new):
        ''' A daughter can over-write this to save more variables if needed ...
        '''
        counter = 0
        for varname, value in self.state.items():
            try:
                if (self.solve_me[varname] == True):
                    tmp1 = Field(np.append(self.state[varname],X_new[counter]),
                            domain = self.state[varname].domain,
                            axis = self.state[varname].axis)
                    self.state[varname] = tmp1 # append with each timestep
                    counter += 1
            except:
                pass

    def compute(self):
        '''Update all diagnostic quantities using current model state.'''
        self.ode_solver()


In [5]:
AA = np.random.rand(19)
aa1 = Field(AA,domain = z_clmn,axis = {'1':z_clmn.axes['depth'].axis_type,'2':z_clmn.axes['depth'].axis_type})
aa1.name='P'

a4 = Chamber_model(state={'P':aa1}, domains=z_clmn, name_inp='a4')

In [6]:
a4.state['P'].append_val(2)

array([ 0.86806255,  0.72517874,  0.36272458,  0.97107452,  0.82461574,
        0.23203389,  0.24943669,  0.40986968,  0.2028324 ,  0.69201971,
        0.16062332,  0.50440383,  0.58126047,  0.95068185,  0.86200981,
        0.59885154,  0.56527016,  0.75993445,  0.01780198,  2.        ])

In [7]:
for dom  in a4.domains.values():
    a =1
dom.axes['depth'].bounds

array([     0.,   1000.,   2000.,   3000.,   4000.,   5000.,   6000.,
         7000.,   8000.,   9000.,  10000.,  11000.,  12000.,  13000.,
        14000.,  15000.,  16000.,  17000.,  18000.,  19000.,  20000.,
        21000.,  22000.,  23000.,  24000.,  25000.,  26000.,  27000.,
        28000.,  29000.,  30000.])

In [8]:
i = 1
j = 2
ynum = 2
xnum = j
if(i==1 or i==ynum and j==xnum):
    print(i,j)

1 2


In [9]:
import numpy as np
from PyMagmaCh.process.diagnostic import DiagnosticProcess
from PyMagmaCh.A1_domain.field import Field

class Iceline(DiagnosticProcess):
    def __init__(self, Tf=-10., **kwargs):
        super(DiagnosticProcess, self).__init__(**kwargs)
        self.time_type = 'diagnostic'
        self.param['Tf'] = Tf

    def find_icelines(self):
        Tf = self.param['Tf']
        Ts = self.state['a1']
        #lat_bounds = self.domains['a1'].axes['lat'].bounds
        self.state['a1'] = self.state['a1']*19.
        noice = np.where(Ts >= Tf, True, False)
        ice = np.where(Ts < Tf, True, False)
        self.diagnostics['noice'] = noice
        self.diagnostics['ice'] = ice
        if ice.all():
            # 100% ice cover
            icelat = np.array([-0., 0.])
        elif noice.all():
            # zero ice cover
            icelat = np.array([-90., 90.])
        else:  # there is some ice edge
            # Taking np.diff of a boolean array gives True at the boundaries between True and False
            boundary_indices = np.where(np.diff(ice.squeeze()))[0] + 1
            #icelat = lat_bounds[boundary_indices]  # an array of boundary latitudes
        #self.diagnostics['icelat'] = icelat


    def compute(self):
        print(self.param['Tf'])
        self.find_icelines()


class StepFunctionAlbedo(DiagnosticProcess):
    def __init__(self, Tf=-10., a0=0.3, a2=0.078, ai=0.62, **kwargs):
        super(DiagnosticProcess, self).__init__(**kwargs)
        self.param['Tf'] = Tf
        self.param['a0'] = a0
        self.param['a2'] = a2
        self.param['ai'] = ai
        sfc = self.domains_var['a1']
        self.add_subprocess('iceline', Iceline(Tf=4, state=self.state))
        self.add_subprocess('iceline2', Iceline(Tf=2, state=self.state))
        self.topdown = False  # call subprocess compute methods first
        self.time_type = 'diagnostic'

    def _get_current_albedo(self):
        '''Simple step-function albedo based on ice line at temperature Tf.'''
        ice = self.subprocess['iceline'].diagnostics['ice']
        # noice = self.subprocess['iceline'].diagnostics['noice']
        albedo = Field(np.where(ice, 1., 0.), domain=self.domains_var['a1'])
        return albedo

    def compute(self):
        self.diagnostics['albedo'] = self._get_current_albedo()



In [10]:
c2 = StepFunctionAlbedo(state=a1)
c2.step_forward()
c2.diagnostics

2
4


/home/tmittal/mantle_plumes/magma_chamber_model/Version2/PyMagmaCh/process/process.py:53: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if (state != None):


{'albedo': Field([ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 'ice': array([ True,  True,  True, False, False, False, False, False, False, False], dtype=bool),
 'noice': array([False, False, False,  True,  True,  True,  True,  True,  True,  True], dtype=bool)}

In [11]:
c2._build_process_type_list()
c2.process_types

{'adjustment': [],
 'diagnostic': [<__main__.Iceline at 0x7fe50204e588>,
 'explicit': [],
 'implicit': []}

In [243]:
#X_init_var = np.array([])
for varname, value in  c2.state.items() :
    X_init_var = np.append(X_init_var,varname)

In [286]:
c2.solve_me ={}
for varname, value in c2.state.items():
            c2.solve_me.update({varname: False})


In [293]:
c2.diagnostics

{'aa': 1,
 'albedo': Field([ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 'ice': Field([ True,  True,  True, False, False, False, False, False, False, False], dtype=bool),
 'noice': Field([False, False, False,  True,  True,  True,  True,  True,  True,  True], dtype=bool)}

In [103]:
def melting_curve_degruyter(T,eta_g,b = 0.5,T_s=973.0,T_l=1223.0):
    """Compute melt fraction-temperature relationship.

    Input:  T is temperature in Kelvin
            eta_g is gas volume fraction
            b is an exponent to approximate composition (1 = mafic, 0.5 = silicic)
            T_s is solidus temperature in Kelvin (Default value = 973 K)
            T_l is liquidus temperature in Kelvin (Default value = 1223 K)
    Output: eta_x,deta_x_dT,deta_x_deta_g (eta_x is crystal volume fraction, others are its derivative with T and eta_g)
    """
    temp1 = T - T_s
    temp2 = T_l - T_s
    eta_x = (1. - eta_g)*(1. - (temp1/temp2)**b)
    deta_x_dT = (1. - eta_g)*(-b*(temp1)**(b-1.)/(temp2)**b)
    deta_x_deta_g = -1.*(1. - (temp1/temp2)**b)
    return eta_x,deta_x_dT,deta_x_deta_g

In [104]:
eta_x,deta_x_dT,deta_x_deta_g = melting_curve_degruyter(1000,.2)